In [ ]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.resnet import preprocess_input, ResNet50, ResNet101
from keras.applications.densenet import preprocess_input, DenseNet121
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout, GlobalAveragePooling2D, Lambda
from tensorflow.keras.callbacks.experimental import BackupAndRestore
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from keras.models import load_model
import IPython.display as ipd
from sklearn.decomposition import PCA
import pickle as pkl

In [ ]:
# Retrieving Visual features files names and labels files names
featureFiles = os.listdir("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Data/")
labelFiles = os.listdir("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Labels/")

for i in range(len(featureFiles)):
  featureFiles[i] = "/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Data/vis_feat_" + str(i+1) + ".npy"
  labelFiles[i] = "/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Labels/vis_labels_" + str(i+1) + ".npy"

trainFiles, trainLabels = featureFiles, labelFiles

print("Total Files = {}".format(len(trainFiles)))

del featureFiles
del labelFiles

Total Files = 5093


In [ ]:
# Custom Generator to load batch files one by one
def customBatchGenerator(featureFiles, labelFiles, batchSize):
  for i in range(batchSize):
    batchX = np.load(featureFiles[i])
    batchY = np.load(labelFiles[i])

    yield batchX, batchY

For Shape (128, 431, 3)

ResNet50

In [ ]:
# ResNet50 Model with pretrained ImageNet weights to extract deep features

# Input Layer
input_layer = Input(shape = (128, 431, 3))
# Preprocess Layer
preprocess = Lambda(lambda x: preprocess_input(x), name='preprocess')(input_layer)
layer_1 = ResNet50(include_top=False, weights="imagenet", input_shape=(128, 431, 3))(preprocess)
# Global Average Pooling Layer
average = GlobalAveragePooling2D()(layer_1)
resnetModel = Model(inputs=input_layer, outputs=average)
resnetModel.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 431, 3)]     0         
                                                                 
 preprocess (Lambda)         (None, 128, 431, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 4, 14, 2048)       23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
length = len(trainFiles)    # Variable to hold total number of files

# Variables to store deep features and labels
features = np.zeros((length*64, 2048))
labels = np.zeros((length*64, 264))

# Calling custom generator to load files one by one
trainBatchGenerator = customBatchGenerator(trainFiles, trainLabels, length)

# Enumerating generator and extracting deep features of various files one by one 
for i, data in enumerate(trainBatchGenerator):
  if i==0:
    features = resnetModel.predict(data[0])
    labels = data[1]
  else:
    features = np.vstack([features, resnetModel.predict(data[0])])
    labels = np.vstack([labels, data[1]])

In [ ]:
print(features.shape)
print(labels.shape)

(325952, 2048)
(325952, 264)


In [ ]:
# Saving extracted deep features and labels into numpy files
np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet50_features_128.npy", features)
np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet50_labels_128.npy", labels)

ResNet101

In [ ]:
# ResNet101 Model with pretrained ImageNet weights to extract deep features

# Input Layer
input_layer = Input(shape = (128, 431, 3))
# Preprocess Layer
preprocess = Lambda(lambda x: preprocess_input(x), name='preprocess')(input_layer)
layer_1 = ResNet101(include_top=False, weights="imagenet", input_shape=(128, 431, 3))(preprocess)
# Global Average Pooling Layer
average = GlobalAveragePooling2D()(layer_1)

resnetModel = Model(inputs=input_layer, outputs=average)
resnetModel.summary()

171458560/171446536 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 431, 3)]     0         
                                                                 
 preprocess (Lambda)         (None, 128, 431, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 4, 14, 2048)       42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 42,658,176
Trainable params: 42,552,832
Non-trainable params: 105,344
_________________________________________________________________


In [ ]:
length = len(trainFiles)    # Variable to hold total number of files

# Variables to store deep features and labels
features = np.zeros((length*64, 2048))
labels = np.zeros((length*64, 264))

# Calling custom generator to load files one by one
trainBatchGenerator = customBatchGenerator(trainFiles, trainLabels, length)

# Enumerating generator and extracting deep features of various files one by one 
for i, data in enumerate(trainBatchGenerator):
  if i==0:
    features = resnetModel.predict(data[0])
    labels = data[1]
  else:
    features = np.vstack([features, resnetModel.predict(data[0])])
    labels = np.vstack([labels, data[1]])

In [ ]:
# Saving extracted deep features into numpy file
np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet101_features_128.npy", features)

DenseNet121

In [ ]:
# DenseNet121 Model with pretrained ImageNet weights to extract deep features
# Input Layer
input_layer = Input(shape = (128, 431, 3))
# Preprocess Layer
preprocess = Lambda(lambda x: preprocess_input(x), name='preprocess')(input_layer)
layer_1 = DenseNet121(include_top=False, weights="imagenet", input_shape=(128, 431, 3))(preprocess)
# Global Average Pooling Layer
average = GlobalAveragePooling2D()(layer_1)

densenetModel = Model(inputs=input_layer, outputs=average)
densenetModel.summary()

29097984/29084464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 431, 3)]     0         
                                                                 
 preprocess (Lambda)         (None, 128, 431, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 4, 13, 1024)       7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 7,037,504
Trainable params: 6,953,856
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
length = len(trainFiles)    # Variable to hold total number of files

# Variables to store deep features and labels
features = np.zeros((length*64, 1024))
labels = np.zeros((length*64, 264))

# Calling custom generator to load files one by one
trainBatchGenerator = customBatchGenerator(trainFiles, trainLabels, length)

# Enumerating generator and extracting deep features of various files one by one 
for i, data in enumerate(trainBatchGenerator):
  if i==0:
    features = densenetModel.predict(data[0])
    labels = data[1]
  else:
    features = np.vstack([features, densenetModel.predict(data[0])])
    labels = np.vstack([labels, data[1]])

In [ ]:
# Saving extracted deep features into numpy file
np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/DenseNet121_features_128.npy", features)

VGG16

In [ ]:
# VGG16 Model with pretrained ImageNet weights to extract deep features

# Input Layer
input_layer = Input(shape = (128, 431, 3))
# Preprocess Layer
preprocess = Lambda(lambda x: preprocess_input(x), name='preprocess')(input_layer)
layer_1 = VGG16(include_top=False, weights="imagenet", input_shape=(128, 431, 3))(preprocess)
# Global Average Pooling Layer
average = GlobalAveragePooling2D()(layer_1)

vggModel = Model(inputs=input_layer, outputs=average)
vggModel.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 431, 3)]     0         
                                                                 
 preprocess (Lambda)         (None, 128, 431, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 4, 13, 512)        14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
length = len(trainFiles)    # Variable to hold total number of files

# Variables to store deep features and labels
features = np.zeros((length*64, 512))
labels = np.zeros((length*64, 264))

# Calling custom generator to load files one by one
trainBatchGenerator = customBatchGenerator(trainFiles, trainLabels, length)

# Enumerating generator and extracting deep features of various files one by one 
for i, data in enumerate(trainBatchGenerator):
  if i==0:
    features = vggModel.predict(data[0])
    labels = data[1]
  else:
    features = np.vstack([features, vggModel.predict(data[0])])
    labels = np.vstack([labels, data[1]])

In [ ]:
# Saving extracted deep features into numpy file
np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/VGG16_features_128.npy", features)